<a href="https://colab.research.google.com/github/s18943/Homework_PAD/blob/master/9_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from scipy import stats
import pandas as pd
from statsmodels.stats.weightstats import ttest_ind
pd.options.display.max_columns = None

In [51]:
mf_income_df = pd.read_csv('PAD_09_PD.csv', sep=';')

In [10]:
mf_income_df.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


### Hipoteza zerowa : nie ma istotnej różnicy w średnich Annual Income pomiędzy kobietami i mężczyznami
### Hipoteza alternatywna : istnieje istotna różnica (bez wskazania kierunku) w średnich Annual Income pomiędzy kobietami i mężczyznami.

In [18]:
mf_income_df.groupby('Gender').describe()

Age                                                       \
        count       mean        std   min    25%   50%   75%   max   
Gender                                                               
Female  112.0  38.098214  12.644095  18.0  29.00  35.0  47.5  68.0   
Male     88.0  39.806818  15.514812  18.0  27.75  37.0  50.5  70.0   

       Annual Income (k$)                                                  \
                    count       mean        std   min    25%   50%    75%   
Gender                                                                      
Female              112.0  59.250000  26.011952  16.0  39.75  60.0  77.25   
Male                 88.0  62.227273  26.638373  15.0  45.50  62.5  78.00   

              Spending Score (1-100)                                        \
          max                  count       mean       std  min   25%   50%   
Gender                                                                       
Female  126.0                  112.0  51.526786  24.11495  5.0  35.0  50.0   
Male    137.0                   88.0  48.511364  27.89677  1.0  24.5  50.0   

                    
         75%   max  
Gender              
Female  73.0  99.0  
Male    70.0  97.0

In [12]:
male = mf_income_df.query('Gender == "Male"')['Annual Income (k$)']
female = mf_income_df.query('Gender == "Female"')['Annual Income (k$)']

## Performing ttest in SciPy

In [29]:
res = stats.ttest_ind(male, female, equal_var=True)
display(res)

Ttest_indResult(statistic=0.795022298602198, pvalue=0.42755249399927864)

## Performing ttest in statsmodels

In [32]:
ttest_ind(male, female)

(0.795022298602198, 0.42755249399927864, 198.0)

#### We managed to revoke the null hypothesis, Justifications:
##### we see a significant difference in Annual Income for men 	(M = 62.23, SD = 26.63) and women (M = 59.25, SD = 26.01)
##### we have p = 0.427, which is greater than 0.05 

## Zadanie 2

In [58]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

for_regression_df = mf_income_df.copy()
for_regression_df.columns = ['Gender', 'Age', 'Annual_Income', 'Spending_Score']

model = smf.ols(formula='Spending_Score ~ Annual_Income + Age + C(Gender)', data=for_regression_df).fit()

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:         Spending_Score   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.095
Method:                 Least Squares   F-statistic:                     7.960
Date:                Mon, 16 May 2022   Prob (F-statistic):           4.91e-05
Time:                        17:29:09   Log-Likelihood:                -922.05
No. Observations:                 200   AIC:                             1852.
Df Residuals:                     196   BIC:                             1865.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            73.9300      6.64

In [71]:
model.pvalues

Intercept            1.287752e-22
C(Gender)[T.Male]    5.671172e-01
Annual_Income        9.050939e-01
Age                  3.055177e-06
dtype: float64

In [70]:
model.bse

Intercept            6.642253
C(Gender)[T.Male]    3.511825
Annual_Income        0.066420
Age                  0.124916
dtype: float64

In [72]:
model.params

Intercept            73.930034
C(Gender)[T.Male]    -2.013234
Annual_Income         0.007929
Age                  -0.600371
dtype: float64

In [83]:

import plotly.express as px
%matplotlib inline

for_heatmap_df = mf_income_df.copy()
design = sm.tools.categorical(mf_income_df['Gender'], drop=True)
for_heatmap_df['Gender'] = design.iloc[:,0]

fig = px.imshow(for_heatmap_df.corr(), text_auto=True)
fig.show()

#### Usuwamy age bo najmniej koreluje z Spending score

In [93]:
for_regression_df = mf_income_df.copy()
for_regression_df.columns = ['Gender', 'Age', 'Annual_Income', 'Spending_Score']

model = smf.ols(formula='Spending_Score ~ Annual_Income - Age + C(Gender)', data=for_regression_df).fit()

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:         Spending_Score   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.3510
Date:                Mon, 16 May 2022   Prob (F-statistic):              0.704
Time:                        17:57:49   Log-Likelihood:                -933.19
No. Observations:                 200   AIC:                             1872.
Df Residuals:                     197   BIC:                             1882.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            50.7565      4.81